# Análise do confronto Cruzeiro x Flamengo

- Quais foram as expectativas antes do jogo?
- E durante o jogo?
- E no fim do jogo?

Outros objetivos:

- Validar os ídolos das torcidas
- Sentimento da torcida (antes, durante e pós)
- Volume de tweets
- Influentes


Autor: **Leonardo Flores**


## Load Data

In [8]:
from pymongo import MongoClient
import pandas as pd
import numpy
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import vincent
from bson import json_util, ObjectId
from pandas.io.json import json_normalize
import json

vincent.core.initialize_notebook()

client = MongoClient('localhost', 27017)
db = client.final
collection = db.tweets.find({"retweeted_status": None}, {
    "_id": 0, 
    "created_at": 1, 
    "text": 1, 
    "unique_id": 1, 
    "retweet_count": 1, 
    "favorite_count": 1, 
    "retweeted": 1, 
    "user.id": 1,
    "user.screen_name": 1,
    "user.name": 1,
    "entities.hashtags": 1,
    "entities.user_mentions.screen_name": 1,
    "entities.user_mentions.id": 1,
    "retweeted_status.created_at": 1,
    "retweeted_status.unique_id": 1,
    "retweeted_status.screen_name": 1,
    "retweeted_status.name": 1
})

def mongo_to_dataframe(mongo_data):
    sanitized = json.loads(json_util.dumps(mongo_data))
    normalized = json_normalize(sanitized)
    df = pd.DataFrame(normalized)

    return df

df = mongo_to_dataframe(collection)

## Normalize Date

In [9]:
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.set_index(['created_at'])
df.index = df.index.tz_localize('UTC').tz_convert('America/Sao_Paulo')
df.index = df.index.tz_localize(None)

## Informações Gerais

### Total de Tweets Coletados

- **Hora de início:** 20:35 (27/09)
- **Hora de termíno da coleta:** ~07:00 (28/09)
- **Horário do Jogo:** 21:45 (27/09)


In [10]:
total_tweets = db.tweets.count()
print("{} tweets coletados desde 20:35 (27/09) até aproximadamente 07:00 (28/09)".format(total_tweets))

776140 tweets coletados desde 20:35 (27/09) até aproximadamente 07:00 (28/09)


### Número de tweets por hora (considerando apenas tweets e não retweets)

In [11]:
history_by_hour = df.groupby(pd.TimeGrouper(freq='H')).size()
area = vincent.Area(history_by_hour)
area.axis_titles(x='Hora', y='Nº Tweets')
area.colors(brew='Spectral')
area.display()

## Antes do Jogo (20:00 ~ 21:45)

Como o pessoal estava se comportando?

### Número de Tweets (by 15 min)

In [19]:
before_df = df.between_time('20:00','21:45')
history_by_15_min = before_df.groupby(pd.Grouper(freq='15Min')).size()

vincent.core.initialize_notebook()
line = vincent.Line(history_by_15_min)
line.axis_titles(x='Hora', y='Nº Tweets')
line.colors(brew='Spectral')
line.display()

## Durante o jogo

### Número de Tweets (by 15 min)

In [13]:
before_df = df.between_time('21:46','23:50')
history_by_15_min = before_df.groupby(pd.Grouper(freq='15Min')).size()

line = vincent.Line(history_by_15_min)
line.axis_titles(x='Hora', y='Nº Tweets')
line.colors(brew='Spectral')
line.display()

## Após o fim do jogo

In [14]:
before_df = df.between_time('23:51','02:00')
history_by_15_min = before_df.groupby(pd.Grouper(freq='15Min')).size()

line = vincent.Line(history_by_15_min)
line.axis_titles(x='Hora', y='Nº Tweets')
line.colors(brew='Spectral')
line.display()